## RAG를 구성하고 Chroma벡터 DB를 활용하여 질의하고 응답을 비교해보자!
- Chroma DB 공식 페이지 : https://www.trychroma.com/

In [2]:
from openai import OpenAI
from getpass import getpass
import re
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
MY_API_KEY = getpass("OpenAI API Key:")

In [4]:
client = OpenAI(api_key=MY_API_KEY)

### gpt-3.5 모델에게 일반적인 질의

In [5]:
completion = client.chat.completions.create(model = "gpt-3.5-turbo",
                                            messages = [{'role':'user', 'content':'기술평가가 뭐야?'}],
                                            temperature = 0
                                           )
print(completion.choices[0].message.content)

기술평가는 특정 기술이나 제품의 성능, 품질, 안전성, 신뢰성 등을 평가하는 과정을 말합니다. 이를 통해 해당 기술이나 제품의 우수성과 개선이 필요한 부분을 파악할 수 있습니다. 주로 실험, 시험, 분석 등의 방법을 사용하여 기술적인 측면을 평가하게 됩니다.


### RAG구조를 활용한 질의

#### 1) 참조 데이터 로드 및 청킹

In [6]:
# 한 번에 여러 개의 PDF 로드
loaders = [PyPDFLoader("data/기술보증기금과 한국경제.pdf"),  # 5 page
           PyPDFLoader("data/기술보증기금 주요업무.pdf")     # 4 page
          ]

ValueError: File path data/기술보증기금과 한국경제.pdf is not a valid file or url

In [5]:
# Recursive splitter 객체 생성
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                              chunk_overlap = 50
                                             )

In [6]:
my_chunks = []   # 청크를 보관할 리스트

for loader in loaders :
    # 2개의 PDF파일에서 추출된 document 객체들을 하나의 리스트에 합쳐줌
    # extend : 리스트에 다른 리스트의 요소들을 추가
    my_chunks.extend(loader.load_and_split(pdf_splitter))

print("청크 개수 :", len(my_chunks))
my_chunks
# 두 PDF파일의 총 page수는 9였지만 chunk_size를 1000으로 청킹을 진행한 결과 총 13개의 청크가 생성됨

청크 개수 : 13


[Document(metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하는 종업원이 1천명 이하이고 총자산액이 1천억원 이하인 기업이 부담하는 금전\n채무 보증. \n기술평가 해당 기술과 관련된 기술성·시장성·사업타당성 등을 종합적으로 평가하여 \n금액·등급·의견 또는 점수 등으로 표시하는 것을 말한다 \n* 기금은 기술평가의 객관성 및 공정성 등을 확보하기 위하여 기술평가의 \n기준·절차·방법·종류 등에 관한 사항을 미리 정하여야 

#### 2) 임베딩 설정

In [7]:
!pip install langchain-chroma

  Using cached langchain_chroma-0.2.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached fastapi-0.115.8-py3-none-any.whl.metadata (27 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.11.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.30.0-

In [9]:
# langchain에서 지원해주는 Chroma 클래스 임포트
from langchain_chroma import Chroma
# langchain과 연동된 openai에서 지원하는 pre-trained Embedding 모델 활용
from langchain_openai import OpenAIEmbeddings

In [10]:
my_embedding = OpenAIEmbeddings(model = "text-embedding-3-small",
                                api_key = MY_API_KEY 
                               )

In [11]:
# 임베딩 예시
my_list = ["오늘 눈이 너무 많이와서 빨리 집에 가면 안될 것 같아요", "늦게까지 남아서 공부합시다^^"]

# embed_documents : 임베딩 진행(문자열이 들어있는 리스트로 입력, 임베딩 차원은 디폴트 1536차원)
temp_embedding = my_embedding.embed_documents(my_list)
temp_embedding

[[0.007044608239084482,
  -0.0017696806462481618,
  -0.07696191966533661,
  0.017841791734099388,
  0.011479469947516918,
  -0.022771311923861504,
  0.03530832752585411,
  -0.02986709214746952,
  -0.07880409061908722,
  -0.03558124229311943,
  -0.047998856753110886,
  0.05956361070275307,
  0.020792681723833084,
  -0.00854137446731329,
  -4.434195943758823e-05,
  -0.038890331983566284,
  -0.010200182907283306,
  0.0003158240288030356,
  0.02613157220184803,
  0.02592688612639904,
  -0.018865222111344337,
  -0.024391742423176765,
  0.003390111029148102,
  -0.008451824076473713,
  0.03136812150478363,
  0.0005479613319039345,
  0.049363426864147186,
  0.012869629077613354,
  0.055913377553224564,
  -0.013807772658765316,
  -0.03170926496386528,
  -0.03660466894507408,
  0.004434862174093723,
  -0.0669323056936264,
  0.01925753615796566,
  0.04980691522359848,
  0.05632274970412254,
  0.05703915283083916,
  -0.06833099573850632,
  -0.02933831885457039,
  0.028485462069511414,
  -0.0317774

In [12]:
np.array(temp_embedding).shape
# 2개의 문자열이 각각 디폴트 값인 1536차원의 벡터로 임베딩 된 것을 확인

(2, 1536)

#### 3) Chroma DB 생성

In [13]:
# DB 생성 경로 설정
my_directory = 'VectorStores/'

# from_documents : 청킹된 문서(document객체)의 정보 및 임베딩, 경로정보로 Chroma DB 생성
vectordb = Chroma.from_documents(documents = my_chunks,
                                 embedding = my_embedding,
                                 persist_directory = my_directory
                                )

### ###한 번 생성된 DB 불러와서 사용하기###
- 프로그램 종료 후 다시 켰을때 코드를 재실행 하게 되면 중첩되어져서 DB에 저장될 것이기 때문에
- 한 번 DB를 생성했다면 생성하는 코드를 다시 실행시키지말고 생성된 DB를 불러와서 사용할 것

In [25]:
my_embedding = OpenAIEmbeddings(model='text-embedding-3-small', 
                                openai_api_key=MY_API_KEY)
my_directory = 'VectorStores/'

# 기존 생성된 db를 불러올 때는 embedding_function에 임베딩 객체 넣기
vectordb = Chroma(persist_directory=my_directory,
                  embedding_function=my_embedding
                 )

#### Chroma 클래스로 자주 사용하는 메소드
- _collection.count : 수집된 벡터의 개수 확인
- _collection.get : 수집된 벡터의 정보 확인
- delete : 특정 id를 가진 벡터 삭제
- add_documents : 여러 문서를 벡터 저장소에 추가
- add_texts : 여러 텍스트를 벡터 저장소에 추가
- add_images : 여러 이미지를 벡터 저장소에 추가
- similarity_search : 유사도 계산을 통해 입력 질의에 대해 가장 유사한 문서 검색

(1) DB 정보 확인

In [14]:
# 수집된 청크 개수 확인(여러번 벡터DB 생성시 값이 계속 누적되어 저장됨)
vectordb._collection.count()

13

In [15]:
# 수집된 청크들의 id, documents 등 정보들이 표시됨
vectordb._collection.get()

{'ids': ['8de443f4-8c07-4978-b4ae-d3af761ccd0b',
  'f2d77dfb-2b4c-4d13-8b01-90d82dba302e',
  '87ecb366-4459-4962-8bb8-89f1bd66c18e',
  '765688e0-b316-4132-a221-188db06a0fb7',
  '6f43bae7-8991-4ce1-ba4b-a348f204fef8',
  '5d5a2caa-2a33-4fa9-a83a-d39e3b40e3eb',
  'c85b372b-3c92-4db7-82f2-ad381b3c8091',
  '4082fb8f-fe84-42e7-91a1-7ca582d10a4d',
  '56997562-670c-4ecf-8897-3c393623a94c',
  '9a2f953e-308a-4949-8b85-fe857eed030f',
  '6f038011-9b4b-43f7-b367-21b5e4939ef2',
  'baec9c78-4fc9-4af8-ba19-4cf98209331f',
  '09a6ab6b-3f0f-4485-bf82-9798288bb593'],
 'embeddings': None,
 'documents': ['페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인

(2) DB에 값 추가

- 텍스트 추가

In [16]:
vectordb.add_texts(texts=["sample_text1", "sample_text2"])
# 추가된 텍스트에 id가 부여되며 표시됨

['240a0d80-aeda-4852-88a8-22d8ac063861',
 'eaecbe0a-fff2-4181-82b0-4f37c0908868']

In [17]:
vectordb._collection.get()['ids']

['8de443f4-8c07-4978-b4ae-d3af761ccd0b',
 'f2d77dfb-2b4c-4d13-8b01-90d82dba302e',
 '87ecb366-4459-4962-8bb8-89f1bd66c18e',
 '765688e0-b316-4132-a221-188db06a0fb7',
 '6f43bae7-8991-4ce1-ba4b-a348f204fef8',
 '5d5a2caa-2a33-4fa9-a83a-d39e3b40e3eb',
 'c85b372b-3c92-4db7-82f2-ad381b3c8091',
 '4082fb8f-fe84-42e7-91a1-7ca582d10a4d',
 '56997562-670c-4ecf-8897-3c393623a94c',
 '9a2f953e-308a-4949-8b85-fe857eed030f',
 '6f038011-9b4b-43f7-b367-21b5e4939ef2',
 'baec9c78-4fc9-4af8-ba19-4cf98209331f',
 '09a6ab6b-3f0f-4485-bf82-9798288bb593',
 '240a0d80-aeda-4852-88a8-22d8ac063861',
 'eaecbe0a-fff2-4181-82b0-4f37c0908868']

- DB에 문서 추가하기

In [19]:
# 새롭게 추가할 문서 2개를 로드 후 청킹
add_loaders = [PyPDFLoader("data/기술보증기금 주요업무_기술보증.pdf"),
               PyPDFLoader("data/기술보증기금 주요업무_기술평가.pdf")
              ]
add_chunks = []
for loader in add_loaders :
    add_chunks.extend(loader.load_and_split(pdf_splitter))
print(len(add_chunks))

15


In [20]:
# 벡터 DB에 문서 추가 명령 실행
vectordb.add_documents(documents=add_chunks)

['e87c250a-372a-45c5-9025-6955c59ee261',
 '27ade535-7eb1-4f76-b7b8-c0ce69200d69',
 '1394c3cf-517e-4bf7-b064-513579d30d69',
 'e5bc01ae-88e1-453c-8722-1bc7e40c60c7',
 '2cb2e1f0-211b-4919-8e32-add50737361e',
 'bdd5276a-b58e-442a-8ed9-b0214ce1dff7',
 '69bbdd86-6b39-4612-8c60-eb5baa2b1334',
 '6d99de66-4570-4856-9354-262779c1aa30',
 'f1c4bac7-02b3-4770-878b-c5d2ccc66101',
 '00f1c488-0359-40cd-bc8d-5ad38c30cb62',
 'f4b47335-a08a-47b9-bac6-98379667f71c',
 '445aadf4-135d-4ee8-bc0e-a625d7b66b1b',
 'e0e481b4-a126-4ad5-b805-67a8522f661b',
 '80da593f-93f8-4d26-a549-9e23e176083b',
 'e8c6d354-c149-4651-b8ac-c4f6bba9878a']

In [21]:
vectordb._collection.count()
# 기존 13개에서 text로 2개 추가하고, 문서로 15개 추가되어서 30개!

30

(3) DB값 삭제

In [22]:
# text로 추가한 값 2개 삭제
vectordb.delete(ids=['240a0d80-aeda-4852-88a8-22d8ac063861',
                     'eaecbe0a-fff2-4181-82b0-4f37c0908868'])
vectordb._collection.count()

28

In [23]:
# 모든 데이터 삭제
vectordb.delete_collection()

In [24]:
vectordb._collection.count()

ValueError: Chroma collection not initialized. Use `reset_collection` to re-create and initialize the collection. 